In [1]:
import os
os.getcwd()

'/Users/leonardo.cavalcante/Documents/Education - Learning/GitHub/OCR_Table'

In [5]:
from pathlib import Path
downloads_path = str(Path.home() / "Downloads")
os.chdir(downloads_path)
os.getcwd()

'/Users/leonardo.cavalcante/Downloads'

##### Let’s import all the libraries that will be used in this article

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [27]:
import pandas as pd
import PIL
import pytesseract
import os
from datetime import datetime

##### If you have non-English text data in your tables you will need to download the suitable tesseract data file from this [GitHub Tesseract-OCR link](https://github.com/tesseract-ocr/tessdata).
##### After you’ve downloaded it, put it in your tessdata_dir folder, if you set it to be a custom folder like me you’ll also need the following line:

In [3]:
os.environ['TESSDATA_PREFIX'] = '/Users/leonardo.cavalcante/Documents/Education - Learning/GitHub/OCR_Table/Languages/'

---
##### To check what languages that pytesseract detected used the following print statement:

In [4]:
print(pytesseract.get_languages())

['eng', 'fra']


---

##### Now let’s set the languages we will use and our config for the pytesseract library:

In [5]:
special_config = '--psm 12 --oem 1'
languages_ = "eng+fra" # For multiple language use like "eng+fra+spa+deu+chi_sim" and so on

##### If you want to read more about the config options, please refer to this [link](https://muthu.co/all-tesseract-ocr-options/).
The above special_config worked best for my personal needs.

Now let’s load the image:

In [6]:
# Any image format will do
image_path = "/Users/leonardo.cavalcante/Documents/Education - Learning/GitHub/OCR_Table/Images_to_Treat/Image.webp"

In [7]:
# You can use opencv for that option too
img_pl=PIL.Image.open(image_path)

##### Now let’s use the function pytesseract.image_to_data(), this function returns verbose data including boxes, confidences, line, and page numbers:

In [8]:
data = pytesseract.image_to_data(
                        img_pl, 
                        lang=languages_, 
                        output_type='data.frame', 
                        config=special_config)

##### Now let’s “optimize” the DataFrame so it will hold only data that is important, I will apply the following:

Take only the columns: left, top, width, text
Sum the columns left and width to create a new column left+width
Sort according to top, reindex columns, and drop None values

In [9]:
def optimizeDf(old_df: pd.DataFrame) -> pd.DataFrame:
    df = old_df[["left", "top", "width", "text"]]
    df['left+width'] = df['left'] + df['width']
    df = df.sort_values(by=['top'], ascending=True)
    df = df.groupby(['top', 'left+width'], sort=False)['text'].sum().unstack('left+width')
    df = df.reindex(sorted(df.columns), axis=1).dropna(how='all').dropna(axis='columns', how='all')
    df = df.fillna('')
    return df

In [10]:
data_imp_sort = optimizeDf(data)

/var/folders/zp/yqzhd4ds5b39vx46_27blb2m0000gp/T/ipykernel_11284/1445764843.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['left+width'] = df['left'] + df['width']


##### Now we have the data in our DataFrame called data_imp_sort, if you followed so far you’ll notice that some of the columns and rows are split (for example a cell has the value of “Happy Birthday Jim” but now you have 3 columns one for each of the words), from what I saw the difference between the columns or the rows are up to 10 pixels so I’ll use that as a threshold but you can change it as you like:
The below functions are merging the columns and rows respectively:

In [11]:
def mergeDfColumns(old_df: pd.DataFrame, threshold: int = 30, rotations: int = 10) -> pd.DataFrame:
  df = old_df.copy()
  for j in range(0, rotations):
    new_columns = {}
    old_columns = df.columns
    i = 0
    while i < len(old_columns):
      if i < len(old_columns) - 1:
        # If the difference between consecutive column names is less than the threshold
        if any(old_columns[i+1] == old_columns[i] + x for x in range(1, threshold)):
          new_col = df[old_columns[i]].astype(str) + df[old_columns[i+1]].astype(str)
          new_columns[old_columns[i+1]] = new_col
          i = i + 1
        else: # If the difference between consecutive column names is greater than or equal to the threshold
          new_columns[old_columns[i]] = df[old_columns[i]]
      else: # If the current column is the last column
        new_columns[old_columns[i]] = df[old_columns[i]]
      i += 1
    df = pd.DataFrame.from_dict(new_columns).replace('0', '').replace(0, '').replace(np.nan, '')
  return df.dropna(axis='columns', how='all').drop_duplicates().reset_index(drop=True)

  # df = pd.DataFrame.from_dict(new_columns).replace('', np.nan).dropna(axis='columns', how='all')
  # return df.replace(np.nan, '')

In [12]:
def mergeDfRows(old_df: pd.DataFrame, threshold: int = 15) -> pd.DataFrame:
    new_df = old_df.drop_duplicates().iloc[:1]
    for i in range(1, len(old_df)):
        # If the difference between consecutive index values is less than the threshold
        #if abs(old_df.index[i] - old_df.index[i - 1]) < threshold: 
        #    new_df.iloc[-1] = new_df.iloc[-1].astype(str) + old_df.iloc[i].astype(str)
        #else: # If the difference is greater than the threshold, append the current row
            # new_df = new_df.append(old_df.iloc[i])
            # df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
            # new_df = pd.concat([new_df, old_df], ignore_index=True).replace(np.nan, '').replace(0, '').drop_duplicates()
        new_df = pd.concat([new_df, old_df], ignore_index=True).replace('0', '').replace(0, '').replace('', np.nan)
    return new_df.drop_duplicates().dropna(axis='rows', how='all').reset_index(drop=True)

In [13]:
df_new_col = mergeDfColumns(data_imp_sort)
merged_row_df = mergeDfRows(df_new_col)

##### Now that the columns and rows are merged according to the threshold, in some of the cases we will still have one or more of the following:
Empty rows and/or columns that hold an empty value (not None but still empty, like an empty string)
Columns that hold only the value of | with or without empty cells (sometimes if the inner borders are not thick enough it may recognize it as a character)
The following function takes care of these scenarios, if you have any additional scenarios you can easily customize the function

In [14]:
def clean_df(df):
    # Remove columns with all cells holding the same value and its length is 0 or 1
    df = df.loc[:, (df != df.iloc[0]).any()]
    # Remove rows with empty cells or cells with only the '|' symbol
    df = df[(df != '|') & (df != '') & (pd.notnull(df))] 
    # Remove columns with only empty cells
    df = df.dropna(axis=1, how='all')
    return df.fillna('')

In [15]:
cleaned_df = clean_df(merged_row_df.copy())
cleaned_df.reset_index(drop=True, inplace=True)

In [44]:
# cleaned_df.to_excel('Outputs/Excel_Scanned.xlsx', index=False, sheet_name='Scanned')

datetime_str = datetime.today().strftime('%Y-%m-%d at %H.%M.%S')
cleaned_df.to_excel('Outputs/' + datetime_str + ' - Excel Scanned.xlsx', index=False, sheet_name='Scanned')

In [52]:
import os
cwd = os.getcwd()
print(cwd)

/Users/leonardo.cavalcante/Documents/Education - Learning/GitHub/OCR_Table
